In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

In [ ]:
datapd=pd.read_csv('/content/drive/MyDrive/PS_20174392719_1491204439457_log.csv')
datapd.shape

(6362620, 11)

In [ ]:
from sklearn.preprocessing import StandardScaler
col=['amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest']
features=datapd[col]
scaler=StandardScaler()
scaler=scaler.fit(features.values)
features=scaler.transform(features.values)
scaled_features=pd.DataFrame(features,columns=col)

data_encode=datapd['type']
dummy=pd.get_dummies(data_encode,drop_first=True)
data_fraud=datapd['isFraud']

In [ ]:
dt=datapd.copy()
cols=['nameOrig','nameDest']

dt[cols]=dt[cols].astype('category')
for i in cols:
  dt[i]=dt[i].cat.codes
know=dt[['nameOrig','nameDest']]
know.isnull().sum()

nameOrig    0
nameDest    0
dtype: int64

In [ ]:
flagged=datapd['isFlaggedFraud']

In [ ]:
new_datapd=pd.concat([dummy,know,scaled_features,flagged],axis=1,ignore_index=False)

In [ ]:
col1=['nameOrig','nameDest']
features1=new_datapd[col1]
scaler1=StandardScaler()
scaler1=scaler1.fit(features1.values)
features1=scaler1.transform(features1.values)
know1=pd.DataFrame(features1,columns=col1)

In [ ]:
new_datapd['nameOrig']=know1['nameOrig']
new_datapd['nameDest']=know1['nameDest']

In [ ]:
x=new_datapd
y=data_fraud
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=1234)

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
print("Class distribution before oversampling:", Counter(y_train))


oversampler = RandomOverSampler(random_state=42)
x_train_resampled, y_train_resampled = oversampler.fit_resample(x_train, y_train)


print("Class distribution after oversampling:", Counter(y_train_resampled))

Class distribution before oversampling: Counter({0: 4448029, 1: 5805})
Class distribution after oversampling: Counter({0: 4448029, 1: 4448029})


In [ ]:
model=RandomForestClassifier()
model.fit(x_train_resampled,y_train_resampled)
y_pred=model.predict(x_test)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_pred,y_test))

[[1906303     538]
 [     75    1870]]


In [ ]:
import joblib
joblib.dump(model,'aml_final.pkl')

['aml_final.pkl']

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_pred,y_test))

0.9996788534702161
